In [1]:
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip install pyyaml==5.1

  Using cached PyYAML-5.1-cp38-cp38-linux_x86_64.whl
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.3.1
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
!pip install pycocotools>=2.0.1

In [25]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
torchvision.__version__

1.7.0+cu101 False


/home/mark/miniconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


'0.8.1+cu101'

In [32]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
import numpy as np
# also could try :: !pip install git+https://github.com/facebookresearch/fvcore.git
#!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
#!pip install -e detectron2_repo

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html


In [4]:
!pip install pillow==7.2

In [5]:
!pip install youtube-dl

In [6]:
import detectron2

In [7]:
from detectron2.utils.logger import setup_logger
setup_logger()

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



<Logger detectron2 (DEBUG)>

In [8]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_config
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [9]:
import os, json, cv2, random
from google.colab.patches import cv2_imshow
import tqdm
import time

In [10]:
# download dataset from github
# make this our dataset !git clone https://github.com/sagecodes/geese-object-detection-dataset.git
!git clone https://github.com/mark-nick-o/camera_object_detection.git
    
from detectron2.data.datasets import register_coco_instances
# on my linux hdrive register_coco_instances("my_dataset_train",{},"/home/mark/pics/pics/train.json","home/mark/pics/pics")
# on my linux hdrive register_coco_instances("my_dataset_vaild",{},"/home/mark/pics/pics/validate.json","home/mark/pics/pics")
# on my linux hdrive register_coco_instances("my_dataset_test",{},"/home/mark/pics/pics/test.json","home/mark/pics/pics")
register_coco_instances("my_dataset_train", {}, "/content/camera_object_detection/train.json", "/content/camera_object_detection")
register_coco_instances("my_dataset_val", {}, "/content/camera_object_detection/validation.json", "/content/camera_object_detection")
register_coco_instances("my_dataset_test", {}, "/content/camera_object_detection/test.json", "/content/camera_object_detection")

# This was the original dataset : register_coco_instances("my_dataset_train", {}, "/content/geese-object-detection-dataset/train.json", "/content/geese-object-detection-dataset")
# This was the original dataset : register_coco_instances("my_dataset_val", {}, "/content/geese-object-detection-dataset/validation.json", "/content/geese-object-detection-dataset")
# bonus labled dataset for getting more metrics on our model
# This was the original dataset : register_coco_instances("my_dataset_test", {}, "/content/geese-object-detection-dataset/test.json", "/content/geese-object-detection-dataset")

In [13]:
mydataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")


[12/29 12:10:00 d2.data.datasets.coco]: Loaded 1310 images in COCO format from /home/mark/pics/pics/train.json


In [18]:
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=mydataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])  # for notebook cv2.imshow(vis.get_image()[:, :, ::-1])

TypeError: imshow() missing required argument 'mat' (pos 2)

In [19]:
test_metadata = MetadataCatalog.get("my_dataset_test")

In [21]:
# uncomment if you want to look at the data here --------> dataset_dicts


In [22]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR,"inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

In [24]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg() # initialize configuration
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")) # Define model
# for coco-pretrained R50-FPN Mask R-CNN model : cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("my_dataset_train",) # <- don't forget comma at end
cfg.DATALOADER.NUM_WORKERS = 0 # e.g. 2 : Can adjust for processes loading
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo (transfer learning!)
# for coco-pretrained R50-FPN Mask R-CNN model : cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"   initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 # Image per batch per GPU
cfg.SOLVER.BASE_LR = 0.001 # e.g. 0.02 : pick a learning rate
cfg.SOLVER.MAX_ITER = 900 # was 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256 # was 32 faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8 # for our dataset we have 8 (cow,sheep,deer,horse,pig,goat,man,woman) in the test we have only 1 geese : How many classes are in your dataset

# Create output directory for training
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Set default trainer with no validation (only have one trainer)
# cfg.DATASETS.TEST = ()
# trainer = DefaultTrainer(cfg)

### Un/comment below to run custom trainer with coco validation ###
cfg.DATASETS.TEST = ("my_dataset_val",) # you can implement a validation loader
cfg.TEST.EVAL_PERIOD = 50
trainer = MyTrainer(cfg)

trainer.resume_or_load(resume=False)
trainer.train()

NameError: name 'get_cfg' is not defined

In [26]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output


In [27]:
!ls ./output/

ls: cannot access './output/': No such file or directory


In [28]:
# Initialize config
cfg = get_cfg()
# Must be same as model trained in. Sense default: COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
# Path to saved model weights (trained model)
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# if using datasets for metadata
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8 # was 1 for geese
# Threshhold to show predictions
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
# create predictor instance to use
predictor = DefaultPredictor(cfg)

NameError: name 'get_cfg' is not defined

In [ ]:
# What does an image look like to the computer?
print(im)
#What does the model output look like not drawn on an image?
print(outputs["instances"].to("cpu"))

In [29]:
# Count objects detected in an image
object_count = outputs["instances"].to("cpu")
print("Detected "+str(len(object_count))+" Geese")

NameError: name 'outputs' is not defined

In [ ]:
# Run test on an image
# from detectron2.utils.visualizer import ColorMode
#test_image_path = '/content/geese-object-detection-dataset/img/01ES4X8G607FY4FP9FV0E3WNNA.png'
# on my drive test_image_path = '/home/mark/pics/pics/how_many_cows.jpg'
test_image_path = "/content/camera_object_detection/cows/CW001.jpeg"
im = cv2.imread(test_image_path)
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.5
    )
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])